In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
from scipy.special import softmax
from sklearn.metrics import precision_recall_curve, auc
import pickle

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
df = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')

In [ ]:
from sklearn.model_selection import train_test_split
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
class TextEncodedDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def create_ds(df, tokenizer):
    labels = df["toxic"].values
    text = df["comment_text"].tolist()
    encodings = tokenizer(text, truncation=True, padding=True)
    ds = TextEncodedDataset(encodings, labels)
    return ds

In [ ]:
train_filt_full, val = train_test_split(df, test_size=0.2, random_state=0)
val_ds = create_ds(val, tokenizer)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=True)

In [ ]:
def evaluate(trainer, val_loader):
    res = trainer.prediction_loop(val_loader, description="Preds")
    logits = res[0]
    labels = res[1]
    pred_probas = softmax(logits, axis=1)[:, 1]
    precision, recall, thresholds = precision_recall_curve(labels, pred_probas)
    auc_score = auc(recall, precision)
    return {"auc_score": auc_score, "pos_samples": labels.sum()}

In [ ]:
n_sample_list = [100, 500 , 2000, 10000, 50000] #, 100000]


d_res = dict()
for n_samples in n_sample_list[:1]:
    print("Now running", n_samples)
    train_filt = train_filt_full.sample(n_samples, random_state=0)
    train_filt, test = train_test_split(train_filt, test_size=0.2, random_state=0)
    train_dataset = create_ds(train_filt, tokenizer)
    test_dataset = create_ds(test, tokenizer)

    training_args = TrainingArguments(
        output_dir='./results',          # output directory
        num_train_epochs=100,              # total number of training epochs
        per_device_train_batch_size=16,  # batch size per device during training
        per_device_eval_batch_size=64,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=10,
    )

    model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

    trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=test_dataset             # evaluation dataset
    )

    trainer.train()
    d_res[n_samples] = evaluate(trainer, val_loader)
     
    with open("results.pkl", "wb") as f:
        pickle.dump(d_res, f)

In [ ]:
d_res

In [ ]:
tests = [
    "Please go home you stupid retard",
    "I think this was very nice!",
    "It's all about not doing things and I kind of like that",
    "I think this is the most stupid shit I have seen"
]
labels = [0] * len(tests)

In [ ]:
encodings = tokenizer(tests, truncation=True, padding=True)
ds = TextEncodedDataset(encodings, labels)

In [ ]:
res = trainer.predict(ds)

In [ ]:
softmax(res.predictions, axis=1)